In [3]:
import numpy as np
import cv2

## The Summary of ZFNet-512

In [6]:
def zfnet_summary(net, layer_type=False, layer_shape=False, 
                    get_flops=False, get_memory=False,
                    get_total_time=False, get_layer_time=False):
    # check if the model is empty
    if net.empty():
        print("Net is empty")
        return
        
    # get the types of layers used in the model
    if layer_type:
        print("\nNet contains:")
        for t in net.getLayerTypes():
            print("\t{} layers of type {}".format(net.getLayersCount(t), t))
        
    
    
    # get the tensor shapes for the loaded model and specified input shape
    if layer_shape:
        layers_ids, in_shapes, out_shapes = net.getLayersShapes([1, 3, 224, 224])
        layers_names = net.getLayerNames()
        print("\nNet layers shapes: ")
        for l in range(len(layers_names)):
            in_num, out_num = len(in_shapes[l]), len(out_shapes[l])
            print("  Layer {} has {} inputs and {} outputs".format(layers_names[l],
                                                                 in_num, out_num))
            for i in range(in_num):
                print("\tinput #{} has shape {}".format(i, in_shapes[l][i].flatten()))
            for i in range(out_num):
                print("\toutput #{} has shape {}".format(i, out_shapes[l][i].flatten()))
            
    # compute the number of FLOPs
    if get_flops:
        print("\ngflops: ", net.getFLOPS((1, 3, 224, 224)) * 1e-9)
    
    # report the amount of memory consumed for storing weights and intermediate tensors
    if get_memory:
        w, b = net.getMemoryConsumption((1, 3, 224, 224))
        print("\nweights (mb):", w * 1e-6, ", blobs (mb):", b * 1e-6)
    
    # perform a forward pass for a mock input
    blob = cv2.dnn.blobFromImage(np.zeros((224, 224, 3), np.uint8), 1, (224, 224))
    net.setInput(blob)
    net.forward()
    
    # report the total time
    if get_total_time:
        total, timings = net.getPerfProfile()
        tick2ms = 1000 / cv2.getTickFrequency()
        print("\ninference (ms): {:2f}".format(total * tick2ms))
    
    # report the per layer inference time
    if get_layer_time:
        layer_names = net.getLayerNames()
        print("\n{: <30} {}".format("LAYER", "TIME (ms)"))
        for (i,t) in enumerate(timings):
            print("{: <30} {:.2f}".format(layer_names[i], t[0] * tick2ms))

In [7]:
model_path = "./model/model.onnx"
zfnet = cv2.dnn.readNetFromONNX(model_path)

In [8]:
zfnet_summary(zfnet, layer_type=True)


Net contains:
	5 layers of type Convolution
	3 layers of type InnerProduct
	2 layers of type LRN
	3 layers of type Pooling
	7 layers of type Relu
	1 layers of type Reshape
	1 layers of type Softmax
	1 layers of type __NetInputLayer__


In [9]:
zfnet_summary(zfnet, layer_shape=True)


Net layers shapes: 
  Layer gpu_0/conv1_1 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1   3 224 224]
  Layer gpu_0/conv1_2 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1  96 109 109]
  Layer gpu_0/norm1_1 has 1 inputs and 1 outputs
	input #0 has shape [  1  96 109 109]
	output #0 has shape [  1  96 109 109]
  Layer gpu_0/pool1_1 has 1 inputs and 1 outputs
	input #0 has shape [  1  96 109 109]
	output #0 has shape [  1  96 109 109]
  Layer gpu_0/conv2_1 has 1 inputs and 1 outputs
	input #0 has shape [  1  96 109 109]
	output #0 has shape [ 1 96 54 54]
  Layer gpu_0/conv2_2 has 1 inputs and 1 outputs
	input #0 has shape [ 1 96 54 54]
	output #0 has shape [  1 256  25  25]
  Layer gpu_0/norm2_1 has 1 inputs and 1 outputs
	input #0 has shape [  1 256  25  25]
	output #0 has shape [  1 256  25  25]
  Layer gpu_0/pool2_1 has 1 inputs and 1 outputs
	input #0 has shape [  1 256  25  25]
	output #0 has sha

In [11]:
zfnet_summary(zfnet, get_flops=True, get_memory=True)


gflops:  3.0579383100000004

weights (mb): 349.002144 , blobs (mb): 19.442111999999998


In [12]:
zfnet_summary(zfnet, get_total_time=True, get_layer_time=True)


inference (ms): 496.575014

LAYER                          TIME (ms)
gpu_0/conv1_1                  39.05
gpu_0/conv1_2                  0.00
gpu_0/norm1_1                  29.99
gpu_0/pool1_1                  4.83
gpu_0/conv2_1                  124.64
gpu_0/conv2_2                  0.00
gpu_0/norm2_1                  2.88
gpu_0/pool2_1                  1.48
gpu_0/conv3_1                  88.30
gpu_0/conv3_2                  0.00
gpu_0/conv4_1                  43.45
gpu_0/conv4_2                  0.00
gpu_0/conv5_1                  114.16
gpu_0/conv5_2                  0.00
gpu_0/pool5_1                  3.38
OC2_DUMMY_0                    0.01
gpu_0/fc6_1                    42.08
gpu_0/fc6_2                    0.00
gpu_0/fc7_1                    1.75
gpu_0/fc7_2                    0.00
gpu_0/pred_1                   0.51
gpu_0/softmax_1                0.05
